In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Venus
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат)
date_launch = Time("2025-04-10", scale="utc").tdb
date_flyby_Venus = Time("2025-10-06", scale="utc").tdb
date_arrival = Time("2034-02-17", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_ven)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_ven, ss_jup)
ss_trans_2, ss_target_2 = ss_ven.apply_maneuver(man_lambert_2, intermediate=True)

In [6]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=106 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=209 * u.deg), color="purple", label="Transfer orbit V-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [7]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=106 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=209 * u.deg), color="purple", label="Transfer orbit M-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [6]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [7]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

-141546276.6814773 km
-47815740.35406876 km
-20699281.245386876 km

[  9.73371338 -25.77452099 -11.17194041] km / s


In [8]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

-79389767.63609861 km
63856959.60367952 km
33757442.170733646 km

[-23.88306866 -24.17518577  -9.36659125] km / s


In [10]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

726553479.5180159 km
-139426955.66543025 km
-77442834.37857845 km

[ 2.62450445 12.31864098  5.21613   ] km / s


In [27]:
nh_tof_1 = date_flyby_Venus - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_ven), = iod.lambert(Sun.k, r_earth, r_ven, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=178.99999996230156>

In [28]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 6.53225889 km / s>

In [29]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-4.47277856,  4.73182149,  0.52394999] km / s>

In [30]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end = (norm(vk_ven - v_ven)).to(u.km / u.s)
V_inf_end

<Quantity 12.72273181 km / s>

In [31]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_vec = (vk_ven - v_ven).to(u.km / u.s)
V_inf_end_vec

<Quantity [-8.23557496,  9.32349436,  2.66751997] km / s>

In [32]:
nh_tof_2 = date_arrival - date_flyby_Venus # время полета Венера - Юпитер

(v0_ven, vk_Jup), = iod.lambert(Sun.k, r_ven, r_jup, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=3056.000000032009>

In [33]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_vec_0 = (norm(v0_ven - v_ven)).to(u.km / u.s)
V_inf_end_vec_0

<Quantity 12.87837728 km / s>

In [34]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0_vec = (v0_ven - v_ven).to(u.km / u.s)
V_inf_end_vec_0_vec

<Quantity [-8.05731733, -7.36911695, -6.82849575] km / s>

In [35]:
# Разница подлетного и отлетного гиперболического избытка скорости

V_gip_diff = V_inf_end - V_inf_end_vec_0
V_gip_diff

<Quantity -0.15564547 km / s>

In [36]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_end_2 = (norm(vk_Jup - v_jup)).to(u.km / u.s)
V_inf_end_2

<Quantity 12.40632372 km / s>

In [37]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_end_2_vec = (vk_Jup - v_jup).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-11.57609018,  -4.34675273,  -1.00833775] km / s>

In [38]:
time_adv = nh_tof_1 + nh_tof_2
time_adv

<TimeDelta object: scale='tdb' format='jd' value=3234.9999999943107>

In [39]:
scalar_EV = r_earth[0]*r_ven[0] + r_earth[1]*r_ven[1] +r_earth[2]*r_ven[2] 
scalar_VJ = r_ven[0]*r_jup[0] + r_ven[1]*r_jup[1] +r_ven[2]*r_jup[2]

In [40]:
fi_EV = (scalar_EV/norm(r_earth)/norm(r_ven))
fi_VJ = (scalar_VJ/norm(r_ven)/norm(r_jup))

In [41]:
np.degrees(np.arccos(fi_EV))

<Quantity 62.46017542 deg>

In [42]:
np.degrees(np.arccos(fi_VJ))

<Quantity 150.08077345 deg>

In [43]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 6.5322589 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 4.89612406 km / s>

In [44]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 12.406324

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

16.590297323610816
17.983782441612995


<Quantity -1.39348512 km / s>

In [45]:
## параметры участка Землия-Венера 

from math import*

to_rad = pi/180

ist_an_1 = 165.4666576 * to_rad
ist_an_2 = 103.6086694 * to_rad

exs_EM = (norm(r_ven)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_ven)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.3602464256284655
Фокальный параметр 98233689.865389 km
Большая полуось 112883415.92869352 km


In [46]:
## параметры участка Венера-Юпитер 

ist_an_1_MJ = 0.1629519 * to_rad
ist_an_2_MJ = 209.2930715 * to_rad

exs_MJ = (norm(r_jup)/norm(r_ven) - 1) / (cos(ist_an_1_MJ) - (norm(r_jup)/norm(r_ven) * cos(ist_an_2_MJ)))
p_MJ = norm(r_ven)*(1+cos(ist_an_1_MJ)*exs_MJ)
a_MJ = p_MJ/(1 - exs_MJ*exs_MJ)

print (f"Эксцентриситет {exs_MJ}")
print (f"Фокальный параметр {p_MJ}")
print (f"Большая полуось {a_MJ}")

Эксцентриситет 0.8418861182613433
Фокальный параметр 197691695.5728774 km
Большая полуось 678821596.3335086 km
